In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np 
import torch
import scipy
import scipy.sparse
from math import prod

In [2]:
def generate_01_sparse_matrix(size, density):
    f, c, k = size
    x = scipy.sparse.rand(f, c*k, density, format='csr')
    x.data[:] = 1
    x = np.array(x.todense()).reshape((f, c, k))
    return x

In [3]:
def get_ones_count(matrix):
    return prod(matrix[np.where(matrix == 1)].shape)

In [4]:
def calc_density(matrix):
    ones = get_ones_count(matrix)
    return ones/prod(matrix.shape)

In [5]:
def density_of_remaining_weights(original_matrix, selection_bitmap):
    not_selected = 1-selection_bitmap
    total_unselected = np.sum(not_selected)
    ignored_weights = np.multiply(not_selected, original_matrix)
    unselected_ones = get_ones_count(ignored_weights)
    return  unselected_ones/ total_unselected 

In [9]:
weight_tensor = generate_01_sparse_matrix(size = (16, 16, 1), density = 0.05)
weight_tensor[np.where(weight_tensor != 0)]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [12]:
def find_densest_subtensor_in_weight_tensor(tensor, filter_bounds, channel_bounds, timeout = 10):
    tensor_cpy = np.copy(tensor)
    tensor_cpy[np.where(tensor_cpy == 0)] = -1
    m = gp.Model('densify')
    m.setParam(GRB.Param.TimeLimit, timeout)
    f_size, c_size, k_size = tensor.shape
    F = m.addVars(f_size, vtype=GRB.BINARY, name='F')
    C = m.addVars(c_size, vtype=GRB.BINARY, name='C')
    Z = m.addVars(f_size, c_size, vtype=GRB.BINARY, name='Z')
    min_filters, max_filters = filter_bounds
    m.addConstr(gp.quicksum([F[i] for i in range(len(F))]) <= max_filters )
    m.addConstr(min_filters <= gp.quicksum([F[i] for i in range(len(F))]))
    min_channels, max_channels = channel_bounds
    m.addConstr(gp.quicksum([C[j] for j in range(len(C))]) <= max_channels )
    m.addConstr(min_channels <= gp.quicksum([C[j] for j in range(len(C))]))
    m.addConstrs((Z[i, j] == gp.and_(F[i], C[j]) for i in range(len(F)) for j in range(len(C))), name='and_constraints')
    m.setObjective(gp.quicksum(Z[i, j]*tensor_cpy[i, j, k] for i in range(len(F)) for j in range(len(C)) for k in range(k_size)), GRB.MAXIMIZE)
    m.optimize()
    dense_filter_indicies = [i for i, f in F.items() if f.X > 0]
    dense_channel_indicies = [j for j, c in C.items() if c.X > 0]
    selection_bitmap = np.zeros(tensor.shape)
    for f in dense_filter_indicies:
        for c in dense_channel_indicies:
            selection_bitmap[f, c] = 1
    dense_tensor = tensor[dense_filter_indicies, :, :][:, dense_channel_indicies, :]
    return dense_tensor, selection_bitmap, dense_filter_indicies, dense_channel_indicies
dense_tensor, selection_bitmap, dense_filter_indicies, dense_channel_indicies = find_densest_subtensor_in_weight_tensor(weight_tensor, (8, 16), (8, 16), timeout = 240)
print(f'density of weight tensor: {calc_density(weight_tensor)}')
print(f'density of dense tensor: {calc_density(dense_tensor)}')
print(f'density of sparse tensor: {density_of_remaining_weights(weight_tensor, selection_bitmap)}')

Set parameter TimeLimit to value 240
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 4224 columns and 256 nonzeros
Model fingerprint: 0x3f1ee2d6
Model has 4096 general constraints
Variable types: 0 continuous, 4224 integer (4224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 2e+01]
Presolve added 4301 rows and 0 columns
Presolve time: 0.01s
Presolved: 4305 rows, 4224 columns, 12749 nonzeros
Variable types: 0 continuous, 4224 integer (4224 binary)
Found heuristic solution: objective -54.0000000

Root relaxation: objective 5.702564e+01, 796 iterations, 0.03 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   57.02564    0  2

In [11]:
dense_tensor, dense_filter_indicies, dense_channel_indicies

(array([[[1.],
         [0.],
         [1.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.]],
 
        [[1.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [1.],
         [0.]],
 
        [[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [1.]],
 
        [[0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.]],
 
        [[0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.]],
 
        [[0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.]],
 
        [[1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [1.]],
 
        [[0.],
         [1.],
         [1.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.]]]),
 [4, 14, 20, 21,